In [1]:
%load_ext autoreload
# %autoreload 2

In [2]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[08-07 13:27:09][INFO] PC| 2025-08-07 13:27:09.663742


In [3]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [4]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[08-07 13:27:18][INFO] 2025-08-07 13:27:18.981612
[08-07 13:27:18][INFO] PC| 2025-08-07 13:27:18.985256
[08-07 13:27:18][INFO] 2025-08-07 13:27:18.985256
[08-07 13:27:18][INFO] PC| 2025-08-07 13:27:18.990691
[08-07 13:27:18][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[08-07 13:27:18][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[08-07 13:27:19][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[08-07 13:27:19][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [5]:
# ####################################################################
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """
    # COMPLETAR
    # Debe basarse en la función gauss_jordan
    pass

In [17]:
def inv_matrix(A):
    """Inversión de una matriz cuadrada mediante Gauss-Jordan sin numpy."""
    n = len(A)
    # Copia profunda de la matriz A y crea la identidad
    AM = [row[:] for row in A]
    I = [[float(i == j) for i in range(n)] for j in range(n)]

    for fd in range(n):  # fd stands for focus diagonal
        # Normalizar la fila fd
        fdScaler = AM[fd][fd]
        if fdScaler == 0:
            # Buscar una fila para intercambiar
            for i in range(fd+1, n):
                if AM[i][fd] != 0:
                    AM[fd], AM[i] = AM[i], AM[fd]
                    I[fd], I[i] = I[i], I[fd]
                    fdScaler = AM[fd][fd]
                    break
            else:
                raise ValueError("La matriz no es invertible.")
        for j in range(n):
            AM[fd][j] /= fdScaler
            I[fd][j] /= fdScaler
        # Hacer ceros en la columna fd para todas las filas menos la fd
        for i in range(n):
            if i == fd:
                continue
            crScaler = AM[i][fd]
            for j in range(n):
                AM[i][j] -= crScaler * AM[fd][j]
                I[i][j] -= crScaler * I[fd][j]
    return I

## Ejemplos
* Ejemplo 1

In [18]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

[[0.5, -0.5, -1.5, 1.5],
 [-0.5, 1.5, -1.5, 0.5],
 [-1.5, -1.5, 3.5, -1.5],
 [1.5, 0.5, -1.5, 0.5]]

* Ejemplo 2

In [19]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

[[-34.0, 31.0, 52.0, -20.0],
 [19.0, -17.0, -29.0, 11.0],
 [12.0, -11.0, -18.0, 7.0],
 [1.0, -1.0, -2.0, 1.0]]

## Ejercicios

* Ejercicio 1

In [20]:
A = [[2, -3], [-1, 1]]
inv_matrix(A)

[[-1.0, -3.0], [-1.0, -2.0]]

* Ejercicio 2

In [21]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

[[-36.0, 45.0, 60.0, -80.0],
 [3.0, -4.0, -5.0, 7.0],
 [9.0, -11.0, -15.0, 20.0],
 [29.0, -36.0, -48.0, 64.0]]

* Ejercicio 3

In [22]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

[[2.0, -1.0, 0.0, -1.0, 0.0, 2.0, 2.0, -1.0],
 [0.0, 1.0, 1.0, -1.0, 0.0, 0.0, 0.0, 0.0],
 [6.0, -1.0, 0.0, -3.0, 1.0, 1.0, 4.0, -3.0],
 [6.0, 1.0, -1.0, -3.0, 1.0, -3.0, 3.0, -3.0],
 [2.0, -1.0, 1.0, -1.0, 0.0, 3.0, 2.0, -1.0],
 [-2.0, 2.0, 0.0, 0.0, 0.0, -4.0, -2.0, 1.0],
 [-1.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 1.0],
 [-2.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 1.0]]

* Ejercicio 4

In [23]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)

[[14.0, -8.0, 9.0, -4.0, 0.0, -4.0, 9.0, 7.0, -8.0, 3.0],
 [-2.0, 2.0, -1.0, 2.0, 1.0, 1.0, -1.0, -1.0, 1.0, -2.0],
 [-27.0, 14.0, -18.0, 5.0, -2.0, 7.0, -17.0, -13.0, 16.0, -2.0],
 [12.0, -6.0, 8.0, -2.0, 1.0, -3.0, 7.0, 6.0, -7.0, 0.0],
 [6.0, -4.0, 4.0, -2.0, 0.0, -2.0, 4.0, 3.0, -3.0, 2.0],
 [18.0, -9.0, 12.0, -4.0, 1.0, -5.0, 11.0, 9.0, -11.0, 1.0],
 [8.0, -4.0, 5.0, -1.0, 1.0, -2.0, 5.0, 4.0, -5.0, 0.0],
 [-5.0, 2.0, -3.0, 0.0, -1.0, 1.0, -3.0, -2.0, 3.0, 1.0],
 [-11.0, 5.0, -7.0, 1.0, -2.0, 2.0, -7.0, -5.0, 7.0, 1.0],
 [1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0]]